# Relative Pose Calibration

In [13]:
import cv2
import numpy as np
import os

In [14]:
# Load your saved intrinsic parameters for camera 1
intrinsic_matrix1 = np.array([[ 1361.57,    0., 956.298],
                           [    0., 1361.57, 546.581],
                           [    0.,    0.,         1.]])
dist_coeffs1 = np.array([[0. , 0. , 0. , 0. , 0.]])

# Load your saved intrinsic parameters for camera 2
intrinsic_matrix2 = np.array([[ 1361.57,    0., 956.298],
                           [    0., 1361.57, 546.581],
                           [    0.,    0.,         1.]])
dist_coeffs2 = np.array([[0. , 0. , 0. , 0. , 0.]])

# Define Charuco board parameters
charuco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
board = cv2.aruco.CharucoBoard((5, 5), 0.08, 0.06, charuco_dict)

# Function to process an image and estimate pose
def estimate_pose(image_path, camera_matrix, dist_coeffs):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    corners, ids, _ = cv2.aruco.detectMarkers(gray, charuco_dict)
    
    if len(corners) > 0:
        ret, charuco_corners, charuco_ids = cv2.aruco.interpolateCornersCharuco(corners, ids, gray, board)
        if charuco_ids is not None and len(charuco_corners) > 3:
            valid, rvec, tvec = cv2.aruco.estimatePoseCharucoBoard(charuco_corners, charuco_ids, board, camera_matrix, dist_coeffs, None, None)
            if valid:
                return rvec, tvec
    return None, None

# Calculate relative pose between two cameras
def calculate_relative_pose(rvec1, tvec1, rvec2, tvec2):
    # Convert rotation vectors to rotation matrices
    R_cam1_to_world, _ = cv2.Rodrigues(rvec1)
    R_cam2_to_world, _ = cv2.Rodrigues(rvec2)
    
    # Calculate rotation matrix from camera 1 to camera 2
    R_cam1_to_cam2 = np.dot(R_cam2_to_world, R_cam1_to_world.T)
    
    # Calculate translation vector from camera 1 to camera 2
    t_cam1_to_cam2 = tvec2 - np.dot(R_cam1_to_cam2, tvec1)
    
    return R_cam1_to_cam2, t_cam1_to_cam2

# Folder paths for images from camera 1 and camera 2
img_folder1 = './cameras/camera1_rgb/'
img_folder2 = './cameras/camera2_rgb/'

# Get list of image paths for camera 1 and camera 2
image_paths1 = [os.path.join(img_folder1, f) for f in os.listdir(img_folder1) if f.endswith('.png')]
image_paths2 = [os.path.join(img_folder2, f) for f in os.listdir(img_folder2) if f.endswith('.png')]

# List to store relative poses
relative_poses = []

# Loop through all image pairs and calculate relative pose
for image_path1, image_path2 in zip(image_paths1, image_paths2):
    # Estimate pose for camera 1
    rvec1, tvec1 = estimate_pose(image_path1, intrinsic_matrix1, dist_coeffs1)
    
    # Estimate pose for camera 2
    rvec2, tvec2 = estimate_pose(image_path2, intrinsic_matrix2, dist_coeffs2)
    
    if rvec1 is not None and rvec2 is not None:
        # Calculate relative pose
        relative_pose = calculate_relative_pose(rvec1, tvec1, rvec2, tvec2)
        relative_poses.append(relative_pose)


In [15]:
# Calculate average relative pose
R_avg = np.zeros((3, 3))
t_avg = np.zeros((3, 1))
for R, t in relative_poses:
    R_avg += R
    t_avg += t

# make sure R_avg is a proper rotation matrix
R_avg /= len(relative_poses)
U, S, Vt = np.linalg.svd(R_avg)
R_avg = np.dot(U, Vt)

t_avg /= len(relative_poses)

# Print average relative pose
print('Average relative rotation:')
print(R_avg)
print('Average relative translation:')
print(t_avg)

Average relative rotation:
[[-0.988019   -0.04995828  0.14602271]
 [ 0.12685739 -0.8017488   0.58404286]
 [ 0.08789575  0.5955695   0.79848063]]
Average relative translation:
[[-0.01313535]
 [-0.62815895]
 [ 0.74457722]]


In [16]:
# calculate the mean error
tot_error = 0
for R, t in relative_poses:
    error = np.linalg.norm(R - R_avg) + np.linalg.norm(t - t_avg)
    print('Error:', error)
    tot_error += error
mean_error = tot_error / len(relative_poses)
print('Mean error:', mean_error)

Error: 0.02707475560353572
Error: 0.07303674634444195
Error: 0.10426462031860617
Error: 0.028504304866996864
Error: 0.07898586339031305
Error: 0.042659631820817445
Error: 0.06647619749288378
Error: 0.058643562887256806
Error: 0.060856547953273214
Error: 0.08861870515310508
Mean error: 0.06291209358312301


In [17]:
import open3d as o3d
import numpy as np

# Define the original rotation matrix and translation vector
R_original = np.eye(3)  # Identity matrix for no rotation
t_original = np.zeros((3, 1))  # Zero translation

# Define the relative rotation matrix and translation vector
R_relative = R_avg.copy()
t_relative = t_avg.copy()

# Create a 4x4 transformation matrix for the original pose
T_original = np.eye(4)
T_original[:3, :3] = R_original
T_original[:3, 3] = t_original.flatten()

# Create a 4x4 transformation matrix for the relative pose
T_relative = np.eye(4)
T_relative[:3, :3] = R_relative
T_relative[:3, 3] = t_relative.flatten()

# Combine the transformations to get the transformed pose
T_transformed = np.dot(T_original, T_relative)

# Create coordinate frames for the original and transformed poses
mesh_frame_original = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.6)
mesh_frame_transformed = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.6)

# Transform the coordinate frames
mesh_frame_original.transform(T_original)
mesh_frame_transformed.transform(T_transformed)

# Visualize both poses
o3d.visualization.draw_geometries([mesh_frame_original, mesh_frame_transformed])

# Simple accumulation of the pointcloud

In [18]:
# Assuming you have these variables already defined
depth_folder1 = './cameras/camera1_depth/'
depth_folder2 = './cameras/camera2_depth/'

depth_paths1 = [os.path.join(depth_folder1, f) for f in os.listdir(depth_folder1) if f.endswith('.png')]
depth_paths2 = [os.path.join(depth_folder2, f) for f in os.listdir(depth_folder2) if f.endswith('.png')]


camera1_intrinsic = o3d.camera.PinholeCameraIntrinsic()
camera1_intrinsic.set_intrinsics(1920, 1080, 1361.57, 1361.57, 956.298, 546.581)
# camera1_intrinsic = o3d.core.Tensor(camera1_intrinsic.intrinsic_matrix, device=o3d.core.Device('cpu:0'))

camera2_intrinsic = o3d.camera.PinholeCameraIntrinsic()
camera2_intrinsic.set_intrinsics(1920, 1080, 1361.57, 1361.57, 956.298, 546.581)
# camera2_intrinsic = o3d.core.Tensor(camera2_intrinsic.intrinsic_matrix, device=o3d.core.Device('cpu:0'))


extrinsics2 = np.eye(4)
extrinsics1 = np.eye(4)
extrinsics1[:3, :3] = R_avg
extrinsics1[:3, 3] = t_avg.flatten()

In [19]:
ratio = 4000
# Create RGBD image
rgbd_image1 = o3d.geometry.RGBDImage.create_from_color_and_depth(
    o3d.geometry.Image(cv2.imread(image_paths1[0])),
    o3d.geometry.Image(cv2.imread(depth_paths1[0])),
    depth_scale=1000 /ratio,
    # depth_trunc=3.0
)

rgbd_image2 = o3d.geometry.RGBDImage.create_from_color_and_depth(
    o3d.geometry.Image(cv2.imread(image_paths2[0])),
    o3d.geometry.Image(cv2.imread(depth_paths2[0])),
    depth_scale=1800 /ratio,
    # depth_trunc=3.0
)

pcd1 = o3d.geometry.PointCloud.create_from_rgbd_image(
    rgbd_image1,
    camera1_intrinsic
)

pcd2 = o3d.geometry.PointCloud.create_from_rgbd_image(
    rgbd_image2,
    camera2_intrinsic
)

# transform point cloud
pcd1.transform(extrinsics1)

# Visualize
o3d.visualization.draw_geometries([pcd1,pcd2])

# TSDF Reconstruction

In [20]:
import open3d as o3d
import open3d.core as o3c

In [21]:
def integrate_two_images(depth_image1, depth_image2, depth_intrinsic_matrix1, depth_intrinsic_matrix2, extrinsics1, extrinsics2, config):
    device = o3d.core.Device(config.device)

    if config.integrate_color:
        vbg = o3d.t.geometry.VoxelBlockGrid(
            attr_names=('tsdf', 'weight', 'color'),
            attr_dtypes=(o3c.float32, o3c.float32, o3c.float32),
            attr_channels=((1), (1), (3)),
            voxel_size=3.0 / 512,
            block_resolution=16,
            block_count=50000,
            device=device)
    else:
        vbg = o3d.t.geometry.VoxelBlockGrid(attr_names=('tsdf', 'weight'),
                                            attr_dtypes=(o3c.float32,
                                                         o3c.float32),
                                            attr_channels=((1), (1)),
                                            voxel_size=3.0 / 512,
                                            block_resolution=16,
                                            block_count=50000,
                                            device=device)


    # Integrate depth information from the first image
    depth1 = o3d.t.io.read_image(depth_image1).to(device)
    depth1 = depth1.to(o3c.float32)

    frustum_block_coords1 = vbg.compute_unique_block_coordinates(depth1, depth_intrinsic_matrix2, extrinsics1,
                                                                 config.depth_scale, config.depth_max)
    vbg.integrate(frustum_block_coords1, depth1, depth_intrinsic_matrix1, extrinsics1, config.depth_scale, config.depth_max)

    # # Integrate depth information from the second image
    # depth2 = o3d.t.io.read_image(depth_image2).to(device)
    # depth2 = depth2.to(o3c.float32)

    # frustum_block_coords2 = vbg.compute_unique_block_coordinates(depth2, depth_intrinsic_matrix2, extrinsics2,
    #                                                              config.depth_scale, config.depth_max)
    # vbg.integrate(frustum_block_coords2, depth2, depth_intrinsic_matrix2, extrinsics2, config.depth_scale, config.depth_max)

    return vbg

In [ ]:
camera1_intrinsic = o3d.core.Tensor(camera1_intrinsic.intrinsic_matrix, device=o3d.core.Device('cpu:0'))
camera2_intrinsic = o3d.core.Tensor(camera2_intrinsic.intrinsic_matrix, device=o3d.core.Device('cpu:0'))

In [24]:
# config(integrate_color, device, depth_scale, depth_max)
from config import ConfigParser
config = ConfigParser()
config.integrate_color = False
config.device = 'cpu:0'
config.depth_scale = 1800 / 4000
config.depth_max = 3.0

# Integrate depth information from the two images
vbg = integrate_two_images(depth_paths1[0], depth_paths2[0], camera1_intrinsic, camera2_intrinsic, extrinsics1, extrinsics2, config)

pcd = vbg.extract_point_cloud()
o3d.visualization.draw([pcd])

mesh = vbg.extract_triangle_mesh()
o3d.visualization.draw([mesh.to_legacy()])

RuntimeError: [1;31m[Open3D Error] (void open3d::t::geometry::PointCloud::SetPointColors(const open3d::core::Tensor&)) /root/Open3D/cpp/open3d/t/geometry/PointCloud.h:173: Tensor has shape {0}, but is expected to have compatible with {None, 3}.
[0;m